In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# 1 Least squares and linear basis functions models
## 1.1 Least squares

In [ ]:
from th.th_costs import compute_loss

def least_squares(y, tx):
    """ calculate the least squares solution.
    
    Parameters
    ----------
    y : the vector of answer values 
    tx : the matrix of input values with augmented 
        first column of 1 to account for constant w0 parameter. 
        Rows are datapoints of D dimensions, 
        and columns are features of N dimensions
    out : mse, w
    """
                    
    # ***************************************************
    # INSERT YOUR CODE HERE
    # least squares: TODO
    
    # There are two methods (both tested and working)
    
    # 1. Manual method with inverse
    # w = np.dot(np.dot(np.linalg.inv(np.dot(tx.T, tx)), tx.T), y)

    # 2. With np solving method, solves x in Ax=b, here we have XˆT*X*w_star = XˆT*y => More robust method because works even if not inversible.
    w = np.linalg.solve(np.dot(tx.T, tx), np.dot(tx.T, y))
    
    mse = compute_loss(y, tx, w)
    return mse, w
    
    # returns mse, and optimal weights
    # ***************************************************
    #raise NotImplementedError

### Load the data
Here we will reuse the dataset `height_weight_genders.csv` from previous exercise section to check the correctness of your implementation. Please compare it with your previous result.

In [ ]:
from th.th_grid_search import generate_w, get_best_parameters, grid_search
from th.th_gradient_descent import gradient_descent

from helpers import *
def test_your_least_squares():
    height, weight, gender = load_data_from_ex02(sub_sample=False, add_outlier=False)
    x, mean_x, std_x = standardize(height)
    y, tx = build_model_data(x, weight)
    # ***************************************************
    # INSERT YOUR CODE HERE
    # least square or grid search: TODO
    
    # First grid search
    # Generate the grid of parameters to be swept
    grid_w0, grid_w1 = generate_w(num_intervals=200) #NB this function creates num_intervals points regularily spaced between -100 and 200 for w0 and -150 and 150 for w1

    # Start the grid search
    grid_losses = grid_search(y, tx, grid_w0, grid_w1)

    # Select the best combinaison
    loss_star_gs, w0_star_gs, w1_star_gs = get_best_parameters(grid_w0, grid_w1, grid_losses)
    print(f"Grid search best loss={loss_star_gs}, w_star params are=[{w0_star_gs}   {w1_star_gs}]\n")
    
    # Now least squares method
    mse_ls, w_star_ls = least_squares(y, tx)
    print(f"Least squares best loss={mse_ls}, w_star params={w_star_ls}\n")
    
    # Now gradient descent method
    mse_gd, w_star_gd = gradient_descent(y, tx, initial_w=[0, 0], max_iters=100, gamma=0.7)
    print(f"Gradient descent best loss={mse_gd[len(mse_gd)-1]}, w_star params={w_star_gd[len(w_star_gd)-1]}\n")

    # this code should compare the optimal weights obtained 
    # by least squares vs. grid search
    # ***************************************************
    #raise NotImplementedError

Test it here

In [ ]:
test_your_least_squares()

## 1.2 Least squares with a linear basis function model
Start from this section, we will use the dataset `dataEx3.csv`.

### Implement polynomial basis functions

In [ ]:
# load dataset
x, y = load_data()
print("shape of x {}".format(x.shape))
print("shape of y {}".format(y.shape))

In [ ]:
def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree.
    
    Output
    ------
    phi_x : matrix formed of augmented features, 
        from x = [x1, x2, x3...].T it returns
        phi_x = [[1, x1, x1ˆ2, x1ˆ3, ..., x1ˆdegree],
                 [1, x2, x2ˆ2, x2ˆ3, ..., x2^degree],
                 ...]
    """
    # ***************************************************
    # INSERT YOUR CODE HERE
    # polynomial basis function: TODO
    # this function should return the matrix formed
    # by applying the polynomial basis to the input data
    
    phi_x = np.empty(shape=(len(x), degree+1))
    for feature_i in range(len(x)):
        for degree_i in range(degree+1):
            phi_x[feature_i][degree_i] = np.power(x[feature_i], degree_i)
            
    return phi_x
    
    # ***************************************************
    #raise NotImplementedError
   
# THEO my test code
build_poly(np.array([2,1, 3]).T, degree=3)

Let us play with polynomial regression. Note that we will use your implemented function `compute_mse`. Please copy and paste your implementation from exercise02.

In [ ]:
from plots import *

def polynomial_regression():
    """Constructing the polynomial basis function expansion of the data,
       and then running least squares regression."""
    # define parameters
    degrees = [1, 3, 7, 12]
    
    # define the structure of the figure
    num_row = 2
    num_col = 2
    f, axs = plt.subplots(num_row, num_col)

    for ind, degree in enumerate(degrees):
        # ***************************************************
        # INSERT YOUR CODE HERE
        # form the data to do polynomial regression.: TODO
        
        tx = build_poly(x, degree)
        
        # ***************************************************
        #raise NotImplementedError
        
        # ***************************************************
        # INSERT YOUR CODE HERE
        # least square and calculate RMSE: TODO
        
        mse_ls, weights = least_squares(y, tx)
        rmse = np.sqrt(2*mse_ls)
        
        # ***************************************************
        #raise NotImplementedError

        print("Processing {i}th experiment, degree={d}, rmse={loss}".format(
              i=ind + 1, d=degree, loss=rmse))
        # plot fit
        plot_fitted_curve(
            y, x, weights, degree, axs[ind // num_col][ind % num_col])
    plt.tight_layout()
    plt.savefig("visualize_polynomial_regression")
    plt.show()

Run polynomial regression

In [ ]:
polynomial_regression()

# 2 Evaluating model predication performance


Let us show the train and test splits for various polynomial degrees. First of all, please fill in the function `split_data()`

In [ ]:
def split_data(x, y, ratio, seed=1):
    """
    split the dataset based on the split ratio. If ratio is 0.8 
    you will have 80% of your data set dedicated to training 
    and the rest dedicated to testing
    """    
    # set seed
    np.random.seed(seed)
    # ***************************************************
    # INSERT YOUR CODE HERE
    # split the data based on the given ratio: TODO
    
    if ratio<0 or ratio>1:
        raise NameError("Ratio is out of [0 1] range")

    nb_data_pts = len(y)
    nb_train_pts = int(np.rint(nb_data_pts*ratio))
    
    train_true_false = np.full(nb_data_pts, False)

    train_pts_indexes = np.random.choice(np.arange(start=0, stop=nb_data_pts), size=nb_train_pts, replace=False) # high is actually one above the max possible integer the function might return
    train_true_false[train_pts_indexes] = True
    
    train_x = x[train_true_false]
    train_y = y[train_true_false]
    
    test_x = x[~train_true_false]
    test_y = y[~train_true_false]
    
    return train_x, train_y, test_x, test_y
    
    # ***************************************************
    #raise NotImplementedError
    
# THEO test my code : 
split_data(x=np.array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [100, 101, 102, 103, 104, 105, 106, 107, 108, 109]]).T, y=np.array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19]).T, ratio=0.7)

Then, test your `split_data` function below.

In [ ]:
from th.th_costs import compute_rmse
from th.th_split_data import split_data

def train_test_split_demo(x, y, degree, ratio, seed):
    """polynomial regression with different split ratios and different degrees."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # split the data, and return train and test data: TODO
    
    train_x, train_y, test_x, test_y = split_data(x, y, ratio, seed)
    
    # ***************************************************
    #raise NotImplementedError
    
    # ***************************************************
    # INSERT YOUR CODE HERE
    # form train and test data with polynomial basis function: TODO
    
    train_x_aug = build_poly(train_x, degree)
    test_x_aug = build_poly(test_x, degree)
    
    # ***************************************************
    #raise NotImplementedError
    
    # ***************************************************
    # INSERT YOUR CODE HERE
    # calculate weight through least square: TODO
    
    train_mse, train_w_ls = least_squares(train_y, train_x_aug)
    
    # ***************************************************
    #raise NotImplementedError
    
    # ***************************************************
    # INSERT YOUR CODE HERE
    # calculate RMSE for train and test data,
    # and store them in rmse_tr and rmse_te respectively: TODO
        
    rmse_tr = np.sqrt(2*train_mse)
    rmse_te = compute_rmse(test_y, test_x_aug, train_w_ls)

    # ***************************************************
    #raise NotImplementedError
    
    print("proportion={p}, degree={d}, Training RMSE={tr:.3f}, Testing RMSE={te:.3f}".format(
          p=ratio, d=degree, tr=rmse_tr, te=rmse_te))


In [ ]:
seed = 6
degrees = [1, 3, 7, 12]
split_ratios = [0.9, 0.5, 0.1]

print("Sorted by proportion")
for split_ratio in split_ratios:
    for degree in degrees:
        train_test_split_demo(x, y, degree, split_ratio, seed)
    print('\n')
    
print("Sorted by degree")
for degree in degrees:
    for split_ratio in split_ratios:
        train_test_split_demo(x, y, degree, split_ratio, seed)
    print('\n')

# 3 Ridge Regression
Please fill in the function below.

In [ ]:
def ridge_regression(y, tx, lambda_):
    """implement ridge regression.
    
    Parameters
    ----------
    in : y, tx, lambda_
    out : w
    
    Return
    ------
    w : computed wheights with ridge regression
    """
    # ***************************************************
    # INSERT YOUR CODE HERE
    # ridge regression: TODO
    
    
    #Very similar code to least_sqares
    
    #There are two methods for solving the algebric system (both tested and working)
    # We want to solve the system Aw_star = b with b = X_T*y
    
    A = np.dot(tx.T, tx) + 2*len(y)*lambda_*np.identity(len(tx.T))
    b = np.dot(tx.T, y)
    
    # 1. Manual method with inverse
    # w = np.dot(np.linalg.inv(A), b)

    # 2. With np solving method, solves x in Ax=b, here we have XˆT*X*w_star = XˆT*y => More robust method because works even if not inversible.
    w = np.linalg.solve(A, b)
    
    return w
    
    
    # ***************************************************
    #raise NotImplementedError
    
# THEOTEST code
test_w = ridge_regression(
    y = np.array([[10, 11, 12, 13, 16, 18, 19]]).T,
    tx = np.array([[  1.,   0.,   0.,   0.],
                  [  1.,   1.,   1.,   1.],
                  [  1.,   2.,   4.,   8.],
                  [  1.,   3.,   9.,  27.],
                  [  1.,   6.,  36., 216.],
                  [  1.,   8.,  64., 512.],
                  [  1.,   9.,  81., 729.]]),  
    lambda_ = 0
)
print("Ridge w =\n", test_w, '\n')

_, test_w = least_squares(
    y = np.array([[10, 11, 12, 13, 16, 18, 19]]).T,
    tx = np.array([[  1.,   0.,   0.,   0.],
                  [  1.,   1.,   1.,   1.],
                  [  1.,   2.,   4.,   8.],
                  [  1.,   3.,   9.,  27.],
                  [  1.,   6.,  36., 216.],
                  [  1.,   8.,  64., 512.],
                  [  1.,   9.,  81., 729.]]))
print("Least squares w=\n", test_w, '\n')

In [ ]:
def ridge_regression_demo(x, y, degree, ratio, seed):
    """ridge regression demo."""
    # define parameter
    lambdas = np.logspace(-5, 0, 15)
    # ***************************************************
    # INSERT YOUR CODE HERE    
    # split the data, and return train and test data: TODO
    
    train_x, train_y, test_x, test_y = split_data(x, y, ratio, seed)

    # ***************************************************
    #raise NotImplementedError
    
    # ***************************************************
    # INSERT YOUR CODE HERE
    # form train and test data with polynomial basis function: TODO
    
    train_x_aug = build_poly(train_x, degree)
    test_x_aug = build_poly(test_x, degree)
    
    # ***************************************************
    #raise NotImplementedError

    rmse_tr = []
    rmse_te = []
    for ind, lambda_ in enumerate(lambdas):
        # ***************************************************
        # INSERT YOUR CODE HERE        
        # ridge regression with a given lambda
        
        w = ridge_regression(train_y, train_x_aug, lambda_)
        
        rmse_tr.append(compute_rmse(train_y, train_x_aug, w))
        rmse_te.append(compute_rmse(test_y, test_x_aug, w))
        
        # ***************************************************
        print("proportion={p}, degree={d}, lambda={l:.3f}, Training RMSE={tr:.3f}, Testing RMSE={te:.3f}".format(
               p=ratio, d=degree, l=lambda_, tr=rmse_tr[ind], te=rmse_te[ind]))
        
    # Plot the obtained results
    plot_train_test(rmse_tr, rmse_te, lambdas, degree)

    #raise NotImplementedError

# THEOTEST code
# ridge_regression_demo(x=np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]).T, 
#                       y=np.array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19]).T,
#                       degree = 3,
#                       ratio = 0.7,
#                       seed = 56)

In [ ]:
seed = 56
degree = 7
split_ratio = 0.5
ridge_regression_demo(x, y, degree, split_ratio, seed)